In [12]:
import pandas as pd
import numpy as np
# eps for making value a bit greater than 0 later on
eps = np.finfo(float).eps
from numpy import log2 as log


In [13]:
df=pd.read_csv(r"C:\Users\jasme\Downloads\weather.csv")
df1=pd.read_csv(r"C:\Users\jasme\Downloads\weather_test.csv")


In [14]:
X_train=df.iloc[:,:-1]
Y_train=df.iloc[:,-1]
X_test=df1.iloc[:,:-1]
Y_test=df1.iloc[:,-1]

In [15]:
#Prior Probabilities [P(yi) for all yi] can be learnt by counting the number of exmples in the training data for each class.
import numpy as np
train_size=X_train.shape[0]
class_priors={}
for outcome in np.unique(Y_train):
    outcome_count = sum(Y_train == outcome)    
    class_priors[outcome] = outcome_count / train_size
print(class_priors)



{0: 0.35714285714285715, 1: 0.6428571428571429}


In [16]:
#Computing likelihoods
features=list(X_train.columns)
likelihoods={}
for outcome in np.unique(Y_train):
    outcome_count = sum(Y_train == outcome)
    for feature in features:
        for feat_value in np.unique(X_train[feature]):
            count=0
            for i in range(len(X_train)):
                if(X_train[feature][i]==feat_value and Y_train[i]==outcome):
                    count=count+1
            likelihoods[(feature,feat_value,outcome)]=(count+1)/(outcome_count+(len(features)))


In [17]:
likelihoods

{('Outlook', 'overcast', 0): 0.1111111111111111,
 ('Outlook', 'rainy', 0): 0.3333333333333333,
 ('Outlook', 'sunny', 0): 0.4444444444444444,
 ('Temp', 'cool', 0): 0.2222222222222222,
 ('Temp', 'hot', 0): 0.2222222222222222,
 ('Temp', 'hot ', 0): 0.2222222222222222,
 ('Temp', 'mild', 0): 0.3333333333333333,
 ('Humidity', '0rmal', 0): 0.2222222222222222,
 ('Humidity', 'high', 0): 0.5555555555555556,
 ('Windy', 'strong', 0): 0.4444444444444444,
 ('Windy', 'weak', 0): 0.3333333333333333,
 ('Outlook', 'overcast', 1): 0.38461538461538464,
 ('Outlook', 'rainy', 1): 0.3076923076923077,
 ('Outlook', 'sunny', 1): 0.23076923076923078,
 ('Temp', 'cool', 1): 0.3076923076923077,
 ('Temp', 'hot', 1): 0.23076923076923078,
 ('Temp', 'hot ', 1): 0.07692307692307693,
 ('Temp', 'mild', 1): 0.38461538461538464,
 ('Humidity', '0rmal', 1): 0.5384615384615384,
 ('Humidity', 'high', 1): 0.3076923076923077,
 ('Windy', 'strong', 1): 0.3076923076923077,
 ('Windy', 'weak', 1): 0.5384615384615384}

In [18]:
#Computing probability for each feature value given each class label in the test examples
a=len(np.unique(Y_train))
prob=np.ones((a,len(X_test)),dtype=np.float)
for outcome in(np.unique(Y_train)):
    outcome_count = sum(Y_train == outcome)
    for feature in features:
        for i in range(len(X_test)):
            if  (feature,X_test[feature][i],outcome) in likelihoods.keys():
                prob[idx][i]=prob[outcome][i]*likelihoods[(feature,X_test[feature][i],outcome)]
            else:
                prob[idx][i]=prob[outcome][i]*(1/(outcome_count+len(features)))


In [19]:
#Multiplying probabilities with class prior probabilities
for i in range(prob.shape[0]):
    prob[i][:]=prob[i][:]*class_priors[i]
 
#Predicting Labels
Y_label=np.zeros(len(Y_test))
for i in range(len(X_test)):
    if (prob[1,i]>=prob[0,i]):
        Y_label[i]=1


In [20]:
from sklearn import metrics
print(metrics.classification_report(Y_test,Y_label))
print(metrics.confusion_matrix(Y_test,Y_label))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

[[0 1]
 [0 1]]


C:\Users\jasme\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jasme\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jasme\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
